In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads 